# __Julia: Arrays and Loops__

<br>

__Finance 5330: Financial Econometrics__ <br>
__Tyler J. Brough__ <br>
__Last Update: February 23, 2021__ <br>

<br>

## __Topics__

1. Array literals
2. Concatenations
3. For loops
4. Comprehensions
5. Element types
6. Dequeues
7. The `bang!` Convention
8. Variable Names vs Copies

<br>

## __Arrays__

Julia has highly efficient multidimensional arrays, both constructed and indexed with square brackets.


Syntax: <br>

```julia
[item1, item2, ...]
```

In [1]:
squares = [1, 4, 9, 15, 25, 36, 49, 64]

8-element Array{Int64,1}:
  1
  4
  9
 15
 25
 36
 49
 64

In [2]:
squares[1]

1

In [3]:
squares[1:3]

3-element Array{Int64,1}:
 1
 4
 9

In [4]:
squares[2:end]

7-element Array{Int64,1}:
  4
  9
 15
 25
 36
 49
 64

In [5]:
squares[4] = 16

16

In [6]:
squares

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

In [7]:
typeof(squares)

Array{Int64,1}

## __Concatenation__

If, instead of commas, you just use spaces, then the values are concatenated horizontally.

In [9]:
cubes = [1, 8, 27, 64, 125, 216, 343, 512]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [10]:
powers = [1:8 squares cubes]

8×3 Array{Int64,2}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

In [11]:
powers[4, 2]

16

In [12]:
powers[:, 3]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [13]:
powers[7, :]

3-element Array{Int64,1}:
   7
  49
 343

In [14]:
powers[3:end, 2:end]

6×2 Array{Int64,2}:
  9   27
 16   64
 25  125
 36  216
 49  343
 64  512

In [15]:
typeof(powers)

Array{Int64,2}

Semicolon separators perform vertical concatenation:

In [18]:
[squares; cubes]

16-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
   1
   8
  27
  64
 125
 216
 343
 512

Whereas commas would simply create an array of arrays:

In [19]:
nested_powers = [[1,2,3,4,5,6,7,8], squares, cubes]

3-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8]
 [1, 4, 9, 16, 25, 36, 49, 64]
 [1, 8, 27, 64, 125, 216, 343, 512]

In [20]:
nested_powers[2][3]

9

Horizontal and vertical concatenation can be used together to as a simple syntax for matrix literals:


In [21]:
[1 3 5
 2 4 6]

2×3 Array{Int64,2}:
 1  3  5
 2  4  6

<br>

## __Loops__

Of course, we could construct this programmatically with a for-loop. The syntax for a `for` loop is

```julia
for *var* in *loop iterable*
    *loop body*
end
```

In [23]:
#A = [] # Allocate an 8x3 matrix to store the values into

A = zeros(8, 3)
for pow in 1:3
    for value in 1:8
        A[value, pow] = value ^ pow
    end
end
A

8×3 Array{Float64,2}:
 1.0   1.0    1.0
 2.0   4.0    8.0
 3.0   9.0   27.0
 4.0  16.0   64.0
 5.0  25.0  125.0
 6.0  36.0  216.0
 7.0  49.0  343.0
 8.0  64.0  512.0

In [24]:
A == powers

true

## __Array Comprehensions__

In [25]:
squares = [value^2 for value in 1:8]

8-element Array{Int64,1}:
  1
  4
  9
 16
 25
 36
 49
 64

In [26]:
cubes = [value^3 for value in 1:8]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [27]:
cubes = [value^3 for value in 1:8]

8-element Array{Int64,1}:
   1
   8
  27
  64
 125
 216
 343
 512

In [28]:
powers = [value^pow for value in 1:8, pow in 1:3]

8×3 Array{Int64,2}:
 1   1    1
 2   4    8
 3   9   27
 4  16   64
 5  25  125
 6  36  216
 7  49  343
 8  64  512

## The element type

Note that every time an array prints out, it is displaying its element type and dimensionality, for example `Array{Int64, 2}`. This describes what it can store — and thus what it can return upon indexing.


In [29]:
eltype(powers)

Int64

In [30]:
typeof(powers[1, 1])

Int64

Further, the array will try to convert any new values assigned into it to its element type:

In [31]:
powers[1, 1] = 1.6

LoadError: InexactError: Int64(1.6)

In [32]:
powers[1, 1] = -5.0 # This can be losslessly converted to an integer

-5.0

In [33]:
powers[1,1]

-5

Arrays that have an exact and concrete element type are generally significantly faster, so Julia will try to find an amenable element type for you in its literal construction syntax:


In [34]:
fortytwosarray = [42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Array{Float64,1}:
 42.0
 42.0
 42.0
 42.0
 42.0
 42.0

In [35]:
for x in fortytwosarray
    show(x)
    println("\tisa $(typeof(x))")
end

42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64
42.0	isa Float64


The `Any` array can be helpful for disabling these behaviors and allowing all kinds of different objects:

In [36]:
anyfortytwos = Any[42, 42.0, 4.20e1, 4.20f1, 84//2, 0x2a]

6-element Array{Any,1}:
   42
   42.0
   42.0
   42.0f0
  42//1
 0x2a

In [37]:
anyfortytwos[1] = "FORTY TWO"
anyfortytwos

6-element Array{Any,1}:
     "FORTY TWO"
   42.0
   42.0
   42.0f0
  42//1
 0x2a

<br>

## Vectors as dequeues

One-dimensional arrays can be appended to and have items removed from them:

In [38]:
fib = [1, 1, 2, 3, 5, 8, 13]

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13

In [39]:
push!(fib, 21)

8-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21

In [40]:
push!(fib, sum(fib[end-1:end]))

9-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13
 21
 34

In [41]:
pop!(fib)

34

In [42]:
pushfirst!(fib, 0)

9-element Array{Int64,1}:
  0
  1
  1
  2
  3
  5
  8
 13
 21

In [43]:
popfirst!(fib)

0

__Aside:__ why so shouty!?

Why are there exclamations in all the above function calls? `push!(fib, ...)`, `pop!(fib)`, etc.?


In [45]:
push!

push! (generic function with 25 methods)

This is entirely a convention. It's a signal to the caller that one of the argument is going to be _mutated_. It's perhaps easiest to demonstrate with an example:


In [46]:
A = rand(0:10, 10)

10-element Array{Int64,1}:
 3
 5
 8
 3
 7
 2
 8
 1
 6
 2

In [47]:
sort(A)

10-element Array{Int64,1}:
 1
 2
 2
 3
 3
 5
 6
 7
 8
 8

In [48]:
A

10-element Array{Int64,1}:
 3
 5
 8
 3
 7
 2
 8
 1
 6
 2

In [49]:
sort!(A)

10-element Array{Int64,1}:
 1
 2
 2
 3
 3
 5
 6
 7
 8
 8

In [51]:
A # That changed A!

10-element Array{Int64,1}:
 1
 2
 2
 3
 3
 5
 6
 7
 8
 8

__Aside:__ names vs. copies

Remember that variables are just names we give our objects. So watch what happens if we give the same object two different names:

In [52]:
fibonacci = [1, 1, 2, 3, 5, 8, 13]

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13

In [53]:
some_numbers = fibonacci
some_numbers[1] = 404
some_numbers

7-element Array{Int64,1}:
 404
   1
   2
   3
   5
   8
  13

In [54]:
fibonacci

7-element Array{Int64,1}:
 404
   1
   2
   3
   5
   8
  13

In [55]:
fibonacci[1] = 1
some_numbers = copy(fibonacci)
some_numbers[2] = 404
fibonacci

7-element Array{Int64,1}:
  1
  1
  2
  3
  5
  8
 13